In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Thesis_Organized

/content/drive/MyDrive/Thesis_Organized


In [3]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6147753 sha256=ada06de8ec641e8755dd6fe99910a37b48f4caab3f4de43a076a3b603aa0a259
  Stored in directory: /tmp/pip-ephem-wheel-cache-5_mrl9pq/wheels/47/e5/15/94c80df2ba85500c5d76599cc307c0a7079d0e221bb6fc4375
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=4fd39de53ebaea504e60699f63f759a923e7a9f56545628d8e154675dc2d6bb9
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=1ba32cb32a42eed5de7dace31eed4ac6be35f0b0e7f148863910a10b198df69c
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built detectron2 fvcore antlr4-python3-runtime


In [4]:
import os
main_dir = os.getcwd()
main_dir

'/content/drive/MyDrive/Thesis_Organized'

In [5]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [7]:
Data_cfg = {
  # COCO TESTING DATASET PATH
  "Coco_labels_train_dir": "/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_train_data.json",
  # COCO TRAINING VALIDATION PATH
  "Coco_labels_val_dir": "/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json",
  # CROPPED IMAGES PATH
  "cropped_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Images_cropped",
  # ORIGINAL IMAGES PATH
  "original_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Original Images",
  # GROUND TRUTH IMAGES PATH
  "GT_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Ground Truth",
  # PATH TO THE CSV FILES OF THE ANNOTATIONS CREATED IN THE DATA PROCESSING PART
  "annotations_path": "Data/Ground Truth/annotations.csv"
}

In [8]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, Data_cfg["Coco_labels_train_dir"], os.path.join(Data_cfg["cropped_Images_dir"], "Train"))
register_coco_instances("my_dataset_val", {}, Data_cfg["Coco_labels_val_dir"], os.path.join(Data_cfg["cropped_Images_dir"], "Val"))

In [9]:
from detectron2.engine.hooks import HookBase
from detectron2.utils.logger import log_every_n_seconds
import detectron2.utils.comm as comm
import time
import datetime
import logging
import numpy as np

class LossEvalHook(HookBase):

    def __init__(self, eval_period, model, data_loader):

        self._model = model
        self._period = eval_period
        self._data_loader = data_loader

    def _do_loss_eval(self):

        total = len(self._data_loader)  # Total number of batches
        num_warmup = min(5, total - 1)  # Number of batches to skip for warm-up

        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []  # Store losses for each batch
        for idx, inputs in enumerate(self._data_loader):
            # Reset timing and loss calculation after warm-up period
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()  # Ensure synchronization in CUDA operations
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            # Log progress and ETA after warm-up period
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    f"Loss on Validation  done {idx + 1}/{total}. {seconds_per_img:.4f} s / img. ETA={eta}",
                    n=5,
                )
            loss_batch = self._get_loss(inputs)  # Compute loss for the current batch
            losses.append(loss_batch)
        mean_loss = np.mean(losses)  # Calculate mean loss
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()  # Synchronize across all processes

        return losses

    def _get_loss(self, data):

        metrics_dict = self._model(data)
        # Ensure all metrics are scalars and detach any tensors from the graph
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())  # Sum up the losses
        return total_losses_reduced

    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [10]:
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators


class CustomTrainer(DefaultTrainer):

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1, LossEvalHook(
            20, # Frequency of calculation - every 20 iterations here
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg, True)
            )
        ))

        return hooks

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        coco_evaluator = COCOEvaluator(dataset_name, output_dir=output_folder)

        evaluator_list = [coco_evaluator]

        return DatasetEvaluators(evaluator_list)

In [11]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode

cfg = get_cfg()
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 2})

In [12]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

In [13]:
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_fpn_backbone', 'FREEZE_AT': 0})

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/06 15:33:14 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/06 15:34:25 detectron2]: Loss on Validation  done 11/97. 0.0001 s / img. ETA=0:00:18
[04/06 15:34:30 detectron2]: Loss on Validation  done 33/97. 0.0001 s / img. ETA=0:00:15
[04/06 15:34:35 detectron2]: Loss on Validation  done 49/97. 0.0001 s / img. ETA=0:00:12
[04/06 15:34:48 detectron2]: Loss on Validation  done 61/97. 0.0001 s / img. ETA=0:00:15
[04/06 15:34:53 detectron2]: Loss on Validation  done 83/97. 0.0001 s / img. ETA=0:00:05
[04/06 15:34:57 d2.utils.events]:  eta: 1:21:10  iter: 3019  total_loss: 0.6098  loss_cls: 0.1197  loss_box_reg: 0.2978  loss_mask: 0.1579  loss_rpn_cls: 0.006824  loss_rpn_loc: 0.03176  validation_loss: 0.6734    time: 2.4902  last_time: 2.5498  data_time: 0.8574  last_data_time: 0.6154   lr: 0.001  max_mem: 9262M
WARNING [04/06 15:35:51 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/06 15:35:51 d2.data.datasets.coco]: Loaded 97 images in COCO format from /content/drive/MyDrive/